<a href="https://colab.research.google.com/github/MudasirIrshad/Agentic_AI_Panaversity_Course/blob/main/ExcelData_for_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install --upgrade --quiet langchain-community unstructured openpyxl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 54.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 27.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.6/167.6 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 61.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.0/211.0 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.2/313.2 kB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.6/114.6 kB 9.3 MB/s eta 0:00:00


In [15]:
!pip install msoffcrypto-tool

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.7/48.7 kB 385.0 kB/s eta 0:00:00


In [65]:

from langchain_core.documents import Document
import pandas as pd
df = pd.read_excel("/content/vector_practice_dataset.xlsx")

data = df['sentence']

documents = []
for i in data:
  documents.append(Document(page_content=i, metadata={'source': 'excel'}))


In [44]:
%pip install -qU langchain-pinecone langchain-google-genai

In [66]:
from google.colab import userdata
pinecone_api_key = userdata.get("PINECONE_API_KEY")


In [67]:
from pinecone import Pinecone
pc = Pinecone(api_key=pinecone_api_key)

In [68]:
from pinecone import ServerlessSpec

index_name = "rag-using-excel-data"  # change if desired

if not pc.has_index(index_name):
    pc.create_index(
        name=index_name,
        dimension=768,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )
else:
    print(f"Index {index_name} already exists. Skipping creation.")

index = pc.Index(index_name)

In [72]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
import os
os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")


In [73]:
from langchain_pinecone import PineconeVectorStore
vector_store = PineconeVectorStore(index=index, embedding=embeddings)

In [74]:
from uuid import uuid4
uuids = [str(uuid4()) for _ in range (len(documents))]
vector_store.add_documents(documents=documents, ids=uuids)

['e7e7dd30-6e9b-4560-be7f-57c0a7cc6bd4',
 '7e607090-b366-4919-a724-092796fa7848',
 '19139cb6-76aa-446b-bd80-4fcf54cb9e01',
 '40d64065-3220-437f-8c9a-7e28a2947c46',
 'b8c31331-bf9e-4b12-ac65-7d019a4e16f7',
 '81905487-5254-45f8-b248-2e0ff909de83',
 'ef3c9cd1-b4b7-478f-8531-a958ea115941',
 '79a89116-35e0-4fae-a5c1-1d1a12a7afd4',
 '8076f077-4f38-4fde-b28f-4d2766064735',
 '17a99190-6ee7-49b4-b404-2ea3b3235f48',
 'ae5fc669-44cf-4947-89ea-4cd4aa61db3e',
 '2e8a82e6-e4f3-4629-861f-74d1db0374c3',
 '2659bc98-5d5a-42ff-aedd-e40a0c8d1741',
 '611ace07-f5f3-4179-bab1-d1f731ae8e6b',
 '4b35cbe6-255a-4baf-84a0-2b510e856471',
 '3a2c55ef-3189-4a43-a245-9d0ff207235d',
 '1cda4b83-0cf4-4d05-b82e-9f0481f1ddc8',
 'c9e0db07-0eea-4a56-b2fa-3e2bd89d7b09',
 'a8f34300-b29e-465b-91db-7725e1a7d7c2',
 '92aa8419-1b30-4724-acad-b76535b20a6b',
 'e3a76790-d679-43c7-859c-7bdc429947f4',
 '3be10656-6a81-41b8-a4e5-b3f6cde59f42',
 'ec065f59-ec06-4245-acad-5cf49fcdbfb7',
 '405f49dd-b8bc-4864-8b3d-c3cacbd07706',
 'bddf3f61-6fde-

In [76]:
result = vector_store.similarity_search("Langchain", k=2)

result

[Document(id='e7e7dd30-6e9b-4560-be7f-57c0a7cc6bd4', metadata={'source': 'excel'}, page_content='LangChain makes it easy to build LLM-powered applications.'),
 Document(id='b8c31331-bf9e-4b12-ac65-7d019a4e16f7', metadata={'source': 'excel'}, page_content='Vector embeddings capture semantic meaning of text.')]